In [1]:
import pandas as pd
import numpy as np

In [7]:
#!curl -L https://www.dropbox.com/s/4hct840aqr9drpl/mturk_surveys.csv?dl=1 -o mturk_surveys.csv


df = pd.read_csv("mturk_surveys.csv")
df["hit_answered_date"]=pd.to_datetime(df["hit_answered_date"])
df['date'] = pd.to_datetime(df['hit_answered_date'].apply(lambda x : x.date()))

# Create a list of all unique worker IDs 
workers = pd.DataFrame(list(set(df.worker_id)), columns=['worker_id']).reset_index().set_index('worker_id')
I = len(workers)

# Calculate the time period of each survey answer. 
# Below we take out time period to be 30-day periods. 
# We can change this to be weeks, months, or anything else
window_length = 30
minDate = min(df['date'])
df['period'] = (df['date'] - minDate).dt.days // window_length 

K = df['period'].max() + 1

data = df.pivot_table(
    index = 'worker_id',
    values = 'period',                    
    aggfunc=('count','min','max')
).join(workers)#.set_index('index').sort_index()

data.head(10)

,count,max,min,index
worker_id,,,,
00008f60155d83e5ac29d6942db234e7,4,31,23,27041
0000ceeb6c91d4a1a53fec0de0cc3460,1,1,1,3220
0001513be5c5e503366d994625ba169e,8,33,20,30621
0001760b118b1695f042928dc8d6eb5c,1,16,16,3616
000314db2adbfc1680937b7a9a7b0564,7,17,17,9475
00033655d33b221f842c11a00064cb05,1,11,11,28515
00038273373747793d09160ede6babdd,1,28,28,44703
000890517a0ee2c855083c5d2d41ad4f,1,0,0,41216
000922f9a9cf32952ea7b526fe655f07,3,19,14,33499


In [8]:
data[data['count'] > (data['max']-data['min'] + 1)].head(10)

,count,max,min,index
worker_id,,,,
000314db2adbfc1680937b7a9a7b0564,7,17,17,9475
00973b532a389cae01e4821b74e5966b,2,10,10,17573
0189e5358390fe1a746c581fd0bac904,3,23,23,40858
01ded1b837424aa6400085f5e34205ef,2,6,6,41414
01fe1039c33407583bb75ecdd9b2cfd9,3,25,24,15365
0282473e990c8b97d93b932ba920025c,2,10,10,30903
02908ecb265c8c77b2d3c5724d222acb,2,20,20,38237
02e3c42db87d78664987b9967ca4d923,2,11,11,31644
03711f3dd781f444764139ee93824c0d,2,7,7,37349
